# TABLE OF CONTENTS

<ul>
    <li><a href='#section_0'>Section 0: Introduction</a></li>
    <li><a href='#section_1'>Section 1: Data Loading</a></li>
    <li><a href='#section_2'>Section 2: Data Preprocessing</a></li>
    <li><a href='#section_3'>Section 3: Data Analysis</a></li>
    <li><a href='#section_4'>Section 4: General Conclusion</a></li>
</ul>

<a id='section_0'></a>

## Section 0: Introduction

### Analysis of Borrowers’ Risk of Defaulting

This is a report for a bank’s loan division in which we determine whether or not a customer’s marital status and number of children has an impact on loan defaulting. The bank already has data on the customers’ history of loan defaulting.

<a id='section_1'></a>

## Section 1: Data Loading 

In [1]:
import pandas as pd
bank_data = pd.read_csv("/datasets/credit_scoring_eng.csv")

bank_data.info()
#bank_data.head(50)
bank_data.tail(50)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21475,0,365213.306266,52,secondary education,1,married,0,F,retiree,1,7718.290,to own a car
21476,0,-2539.502794,52,bachelor's degree,0,married,0,M,business,0,84392.453,transactions with my real estate
21477,0,-1628.432350,54,secondary education,1,civil partnership,1,F,employee,0,11477.425,having a wedding
21478,1,-921.332705,28,Bachelor's Degree,0,married,0,F,employee,0,28989.738,to own a car
21479,0,-685.565994,26,secondary education,1,married,0,M,employee,0,27440.696,buying property for renting out
21480,0,-8799.354717,45,bachelor's degree,0,unmarried,4,F,employee,0,23511.207,housing renovation
21481,0,-829.519627,30,some college,2,civil partnership,1,M,employee,0,70475.341,to buy a car
21482,1,-865.081118,25,secondary education,1,married,0,F,business,0,15050.405,to own a car
21483,2,-2917.035232,38,primary education,3,civil partnership,1,F,employee,1,12386.810,real estate transactions
21484,3,-968.718123,35,secondary education,1,married,0,M,employee,0,15306.792,housing


### Conclusion

- From the info() method we learn:

    - Total observations appear to number 21525.
    - Two columns ('days_employed' & 'total_income') only contain 19351 values [as they are not the main value of interest and only about 10% of the values are missing, we can fill them in].
    - Two pairs of columns (('education' & education_id') & ('family_status' & 'family_status_id')) appear to be duplicates.
    - Two columns have floating point data when integer data would be more appropriate ('days_employed' & 'total_income').
    
- From the head(20) & tail(20) methods we learn:

    - 'days_employed' column has negative values (when not NaN) for all except for retirees.
    - 'education' has mixed capitalization, leading to duplicate categories.
    - 'purpose' has duplicates (e.g., 'purchase of a car', 'buying my own car', 'to buy a car', 'car purchase', & 'second-hand car purchase').
    
- From problem description we learn:

    - Marital status (i.e., 'family_status'/'family_status_id') & number of children (i.e., 'children') are the two most important factors for consideration.
    - "You’ll need to find out if a customer’s marital status and number of children has an impact on whether they will default on a loan. The bank already has some data on customers’ credit worthiness." <-- the debt metric (a boolean value) indicates whether or not they have previously defaulted on a loan
    - 'total_income' represents monthly income
    

<a id='section_2'></a>

## Section 2:<a id='section_0'></a> Data Preprocessing

### Filling Missing Data

In [2]:
# ------------------------------
# IDENTIFY AND FILL MISSING DATA
print("Before:")
print(bank_data.isna().sum())

# Repair 'days_employed' column
# make negative days_employed values positive
def abs(num):
    if num < 0:
        return num * -1
    else:
        return num
    
bank_data['days_employed'] = bank_data['days_employed'].apply(abs)

# calculate median
days_employed_median = bank_data['days_employed'].median()

# replace NaN with median
bank_data['days_employed'] = bank_data['days_employed'].fillna(days_employed_median)

# Repair 'total_income' column
# calculate median
total_income_median = bank_data['total_income'].median()

# replace NaN with median
bank_data['total_income'] = bank_data['total_income'].fillna(total_income_median)

# check again to ensure all missing values have been dealt with
print("\nAfter:")
print(bank_data.isna().sum())

# delete children < 0 and children > 19 observations (found during the categorization step)
row_indexes = bank_data[(bank_data['children'] < 0) | (bank_data['children'] > 19)].index
for index in row_indexes:
    bank_data = bank_data.drop(index)
bank_data = bank_data.reset_index(drop=True)

Before:
children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

After:
children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64


### Conclusion

- Missing data found:
    - 'days_employed' column and 'total_income' column were each missing nearly 1/10 of their observations (represented as NaN).
    - One possible explanation for their absence that the person may have had no verifiable work history (performed work as a freelancer and had no method to show source of income); or perhaps as a freelancer, their income is unstable as is their work schedule and thus unquantifiable under this schema.
    - As neither column was the primary data for the research objective, I filled in all missing values with the column's median.
        - I chose the median because extremely lengthy employment/unemployment or drastic differences in income could have greater impacts on the mean than the median.
    - There were problems, though:
        - Days employed had negative values, which could have been there to indicate 'days unemployed', but was not explicitly mentioned in any of the introductory material for this project; furthermore, negative days employed were paired with positive total income (which could be welfare/unemployment income), but seems unlikely given the quantity of income & the purposes stated for the loan; lastly, there is even a category in the 'income_type' column for unemployed, but this was not always associated with a negative number.

### Data type replacement

In [3]:
# -------------------------------------
# REPLACE REAL NUMBER DATA WITH INTEGER
bank_data['days_employed'] = bank_data['days_employed'].astype('int')
bank_data['total_income'] = bank_data['total_income'].astype('int')
bank_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21402 entries, 0 to 21401
Data columns (total 12 columns):
children            21402 non-null int64
days_employed       21402 non-null int64
dob_years           21402 non-null int64
education           21402 non-null object
education_id        21402 non-null int64
family_status       21402 non-null object
family_status_id    21402 non-null int64
gender              21402 non-null object
income_type         21402 non-null object
debt                21402 non-null int64
total_income        21402 non-null int64
purpose             21402 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


### Conclusion

 - Here, I modified and saved the two columns that were floating point values using the astype method with 'int' parameter. After I checked to confirm that all had been successfully converted.

### Processing duplicates

In [4]:
# -----------------
# DELETE DUPLICATES
# check to see if number of ids equals unique tags
print("----------------------------------------")
print("Education labels are equal to IDs:", len(bank_data['education'].unique()) == len(bank_data['education_id'].unique()))
# not equal, thus make lowercase
bank_data['education'] = bank_data['education'].str.lower()

# check to see if number of ids equals unique tags 
print("Marital status labels are equal to IDs:", len(bank_data['family_status'].unique()) == len(bank_data['family_status_id'].unique()))
# equal, thus do nothing

# check to see if categories are all unqiue 
print("Unique income type labels:", bank_data['income_type'].unique())
# no overlapping categories, thus do nothing

# unify 'purpose' columns by keyword
print("\n----------------------------------------")
print("Value counts for purpose category preprocessing:")
print("Before processing:")
print(bank_data['purpose'].value_counts())
# many overlapping keywords, condense to 
purpose_categories = ['wedding', 'car', 'real estate', 'education']

def change_category(text):
    # contains category name
    for category in purpose_categories:
        if category in text:
            return category
    # exceptions not containing category names - not enough to merit lemmatization or stemming
    if 'univeristy' or 'educated' in text:
        return 'education'
    else:
        return 'real estate'
    return text

bank_data['purpose'] = bank_data['purpose'].apply(change_category) 
# check diff value counts
print("\nAfter processing:")
print(bank_data['purpose'].value_counts())
# only the four categories we want remain 

# check num duplicates
print("\n----------------------------------------")
print("Number of duplicates:", bank_data.duplicated().sum())
# check ratio of duplicate to total number of observations of that category
ind = 0
vals = [['education', 0], ['real estate', 0], ['car', 0], ['wedding', 0]]
for dup in bank_data.duplicated():
    if dup:
        entry = bank_data.loc[ind, 'purpose']
        # reversing indexes so that it matches value countes
        if entry == purpose_categories[0]:
            vals[3][1] += 1
        elif entry == purpose_categories[1]:
            vals[2][1] += 1
        elif entry == purpose_categories[2]:
            vals[1][1] += 1
        else:
            vals[0][1] += 1
    ind += 1

print("\n----------------------------------------")
print('Percentage of duplicates to category counts:')
print('Education = {:.2%}'.format(vals[0][1] / bank_data[bank_data['purpose'] == 'education']['purpose'].count()))
print('Real Estate = {:.2%}'.format(vals[1][1] / bank_data[bank_data['purpose'] == 'real estate']['purpose'].count()))
print('Car = {:.2%}'.format(vals[2][1] / bank_data[bank_data['purpose'] == 'car']['purpose'].count()))
print('Wedding = {:.2%}'.format(vals[3][1] / bank_data[bank_data['purpose'] == 'wedding']['purpose'].count()))

# many duplicates (409) after unifying purpose column
# but cannot drop without loss of generality as the ratios are off compared to entire column's value counts

----------------------------------------
Education labels are equal to IDs: False
Marital status labels are equal to IDs: True
Unique income type labels: ['employee' 'retiree' 'business' 'civil servant' 'unemployed'
 'entrepreneur' 'student' 'paternity / maternity leave']

----------------------------------------
Value counts for purpose category preprocessing:
Before processing:
wedding ceremony                            796
having a wedding                            772
to have a wedding                           769
real estate transactions                    673
buy commercial real estate                  661
buying property for renting out             651
housing transactions                        648
transactions with commercial real estate    646
housing                                     642
purchase of the house                       641
purchase of the house for my family         640
property                                    632
construction of own property             

### Conclusion

- For the duplicates, I performed the following steps:

    - I looked for intracolumn exact duplication for strings:
        - For education and family, there were redundant columns so we could check the unique values of each against each other
            - Education had discrepancies, but converting all entries to lowercase removed the discrepancies; family status had no discrepancies bewteen its redundant columns
        - Income type had no overlapping categories, so processing was unnecessary
    - Next, I looked at the purpose column and noticed that there were myriad ways of saying what amounted to the same four categories
        - Thus, I created a function to modify the purpose column into one of four terms depending on the content of the original statement of purpose
    - When all duplicates of this nature were removed, nearly 409 exact (i.e., all columns of 2+ observations were equal) duplicates appeared in the data
        - I did not remove them because the ratio of duplicates / category varied somewhat among the four categories (e.g., approx. 2.6% of education entries were duplicates, but only half as many were duplicates in real estate and wedding, and car was a mere quarter)
        

### Categorizing Data

In [5]:
# ---------------
# CATEGORIZE DATA
# change confusing column names (necessary?)
bank_data.set_axis(['num_children', 'days_employed', 'age', 'education', 'education_id',
       'marital_status', 'marital_status_id', 'gender', 'income_type', 'prev_default',
       'monthly_income', 'purpose'], axis='columns', inplace=True)

# make broader categories for monthly_income
def income_category(income):
    if income < 10000:
        return "low income"
    if income < 30000:
        return "medium income"
    return "high income"

bank_data['income_group'] = bank_data['monthly_income'].apply(income_category)

# isolating variables of interest (primary questions of kids-default & marital-default)
total_default = bank_data['prev_default'].sum()
relevant_1_data = bank_data[['num_children', 'marital_status_id', 'prev_default']]
marital_status_dict = bank_data[['marital_status', 'marital_status_id']].drop_duplicates().reset_index(drop=True)

pivot_table_1_a = bank_data.pivot_table(index='num_children', values='prev_default', aggfunc='sum')
pivot_table_1_a['ratio'] = pivot_table_1_a['prev_default'] / total_default
pivot_table_1_b = bank_data.pivot_table(index='marital_status', values='prev_default', aggfunc='sum')
pivot_table_1_b['ratio'] = pivot_table_1_b['prev_default'] / total_default

print("----------------------------------------")
print("Primary variables of interest: number of children and marital status")
print(pivot_table_1_a.sort_values('prev_default',ascending=False))
print(pivot_table_1_b.sort_values('prev_default',ascending=False))

total_couples_default = bank_data[(bank_data['prev_default'] == 1) & ((bank_data['marital_status'] == 'married') | (bank_data['marital_status'] == 'civil partnership'))]['prev_default'].count()
childless_couples_default = bank_data[((bank_data['num_children'] == 0) & (bank_data['prev_default'] == 1)) & ((bank_data['marital_status'] == 'married') | (bank_data['marital_status'] == 'civil partnership'))]['prev_default'].count()
print("\nChildless couples who defaulted:", childless_couples_default)
print("Percent of childless couples who defaulted: {:.2%}".format(childless_couples_default / total_couples_default))
total_singles_default = bank_data[(bank_data['prev_default'] == 1) & ((bank_data['marital_status'] == 'unmarried') | (bank_data['marital_status'] == 'divorced') | (bank_data['marital_status'] == 'widow / widower'))]['prev_default'].count()
childless_singles_default = bank_data[((bank_data['num_children'] == 0) & (bank_data['prev_default'] == 1)) & ((bank_data['marital_status'] == 'unmarried') | (bank_data['marital_status'] == 'divorced') | (bank_data['marital_status'] == 'widow / widower'))]['prev_default'].count()
print("\nChildless singles who defaulted:", childless_singles_default)
print("Percent of childless singles who defaulted: {:.2%}".format(childless_singles_default / total_singles_default))

total_default = total_couples_default + total_singles_default
total_childless_default = childless_couples_default + childless_singles_default
print("\nPercent of childless borrowers who defaulted: {:.2%}".format(total_childless_default / total_default))

# isolating variables of interest (secondary questions of income-default & purpose-default)
relevant_2_data = bank_data[['income_group', 'purpose', 'prev_default']]
pivot_table_2_a = bank_data.pivot_table(index='income_group', values='prev_default', aggfunc='sum')
pivot_table_2_a['ratio'] = pivot_table_2_a['prev_default'] / total_default
pivot_table_2_b = bank_data.pivot_table(index='purpose', values='prev_default', aggfunc='sum')
pivot_table_2_b['ratio'] = pivot_table_2_b['prev_default'] / total_default

print("\n----------------------------------------")
print("Secondary variables of interest: income group and loan purpose")
print(pivot_table_2_a.sort_values('prev_default',ascending=False))
print(pivot_table_2_b.sort_values('prev_default',ascending=False))

----------------------------------------
Primary variables of interest: number of children and marital status
              prev_default     ratio
num_children                        
0                     1063  0.613741
1                      444  0.256351
2                      194  0.112009
3                       27  0.015589
4                        4  0.002309
5                        0  0.000000
                   prev_default     ratio
marital_status                           
married                     927  0.535219
civil partnership           385  0.222286
unmarried                   273  0.157621
divorced                     84  0.048499
widow / widower              63  0.036374

Childless couples who defaulted: 745
Percent of childless couples who defaulted: 56.78%

Childless singles who defaulted: 318
Percent of childless singles who defaulted: 75.71%

Percent of childless borrowers who defaulted: 61.37%

----------------------------------------
Secondary variables of int

### Conclusion

- Unexpected anomalies:

    - In the process of completing this step, I found that there was one observation with -1 children. I went back to investigate and it turns out there are 47 such observations, which is small enough to likely be negligible. I went back to delete them.
    - I also found the prospect of several families with 20 children to be implausible. Going back to investigate, I found there were 76 such observations and deleted them as well.

- Categorization:

    - I created a new column for income group, as there were too many different values for the column
    - I then created four pivot tables (two for the primary research questions & two for the secondary ones)

<a id='section_3'></a>

## Section 3: Data Analysis

- Is there a relation between having kids and repaying a loan on time?

Approx. 61% of defaults on loans came from childless borrowers, meaning there appears to be a greater chance to default on a loan if the borrower has no children.

- Is there a relation between marital status and repaying a loan on time?

From the raw numbers, people who are married or in civil partnerships tend to default the most (accounting for roughly approx. 76% of all defaults); those without partners tend to default the least (accounting for the remaining approx. 24%). But we see that only approx. 57% of childless couples defaulted, whereas, approx. 75% of childless singles defaulted, thus hinting at insights. Furthermore, we need to first discover how many borrowers were couples vs. singles, for if over 1200 couples defaulted on their loans, but they account for over 20,000 of the total borrowers, then 1200 doesn't seem like as large a number anymore. [More on this in the General Conclusion]

- Is there a relation between income level and repaying a loan on time?

Yes. Those in the medium income bracket (10000 <= monthly_income < 30000) tend to default the most (approx. 72% of all defaults); whereas those of the lowest income bracket (monthly_income < 10000) tend to default the least (approx. 3% of all defaults). However, this could be due to most low income loan-seekers being denied loans by banks, and thus they only account for a small fraction of borrowers.

- How do different loan purposes affect on-time repayment of the loan?

Those seeking loans for education tend to default the most (approx. 47% of all defaults); whereas those for weddings default the least (approx. 11% of all defaults).

<a id='section_4'></a>

## Section 4: General conclusion

In [6]:
def get_totals(data, group):
    group_total = bank_data[bank_data['marital_status'].isin(group)]
    group_default = bank_data[(bank_data['marital_status'].isin(group))
                 & (bank_data['prev_default'] == 1)]
    return len(group_total), len(group_default)


def couples_default_status(data, group, purpose):
    defaulted = data[(data['marital_status'].isin(group)) 
                 & (data['prev_default'] == 1) & (data['purpose'] == purpose)]
    purpose_total = data[(data['marital_status'].isin(group))
                 & (data['purpose'] == purpose)]
    return len(defaulted), len(purpose_total)



# Total loans sought for couples
couples = ['married', 'civil partnership']
couple_total, couple_default = get_totals(bank_data, couples)
total = len(bank_data)

print("----------------------------------------")
print("STATS FOR COUPLES")
print("Total loans sought by couples:", couple_total)
print("Ratio of loans sought by couples to total loans sought: {:.2%}".format(couple_total / total))
print("Ratio of default by couples to loans sought by couples: {:.2%}".format(couple_default / couple_total))


# Analysis of ratio of defaulting on loans (in order of decreasing financial burden)
defaulted, purpose_total = couples_default_status(bank_data, couples, 'real estate')
print("\nTotal real estate loans sought: {:.2%}".format(purpose_total / couple_total))
print("Defaulted on real estate loan: {:.2%}".format(defaulted / purpose_total))
print("Did not default on real estate loan: {:.2%}".format((purpose_total-defaulted) / purpose_total))

defaulted, purpose_total = couples_default_status(bank_data, couples, 'education')
print("\nTotal education loans sought: {:.2%}".format(purpose_total / couple_total))
print("Defaulted on education loan: {:.2%}".format(defaulted / purpose_total))
print("Did not default on education loan: {:.2%}".format((purpose_total-defaulted) / purpose_total))

defaulted, purpose_total = couples_default_status(bank_data, couples, 'car')
print("\nTotal car loans sought: {:.2%}".format(purpose_total / couple_total))
print("Defaulted on car loan: {:.2%}".format(defaulted / purpose_total))
print("Did not default on car loan: {:.2%}".format((purpose_total-defaulted) / purpose_total))

defaulted, purpose_total = couples_default_status(bank_data, couples, 'wedding')
print("\nTotal wedding loans sought: {:.2%}".format(purpose_total / couple_total))
print("Defaulted on wedding loan: {:.2%}".format(defaulted / purpose_total))
print("Did not default on wedding loan: {:.2%}".format((purpose_total-defaulted) / purpose_total))



# Total loans sought for singles
singles = ['unmarried', 'divorced', 'widow / widower']
single_total = bank_data[bank_data['marital_status'].isin(singles)]
single_default = bank_data[(bank_data['marital_status'].isin(singles))
                 & (bank_data['prev_default'] == 1)]
single_total, single_default = get_totals(bank_data, singles)
total = len(bank_data)

print("\n----------------------------------------")
print("STATS FOR SINGLES")
print("Total loans sought by singles:", single_total)
print("Ratio of loans sought by singles to total loans sought: {:.2%}".format(single_total / total))
print("Ratio of default by singles to loans sought by singles: {:.2%}".format(single_default / single_total))

# Analysis of ratio of defaulting on loans (in order of decreasing financial burden)
defaulted, purpose_total = couples_default_status(bank_data, singles, 'real estate')
print("\nTotal real estate loans sought: {:.2%}".format(purpose_total / single_total))
print("Defaulted on real estate loan: {:.2%}".format(defaulted / purpose_total))
print("Did not default on real estate loan: {:.2%}".format((purpose_total-defaulted) / purpose_total))

defaulted, purpose_total = couples_default_status(bank_data, singles, 'education')
print("\nTotal education loans sought: {:.2%}".format(purpose_total / single_total))
print("Defaulted on education loan: {:.2%}".format(defaulted / purpose_total))
print("Did not default on education loan: {:.2%}".format((purpose_total-defaulted) / purpose_total))

defaulted, purpose_total = couples_default_status(bank_data, singles, 'car')
print("\nTotal car loans sought: {:.2%}".format(purpose_total / single_total))
print("Defaulted on car loan: {:.2%}".format(defaulted / purpose_total))
print("Did not default on car loan: {:.2%}".format((purpose_total-defaulted) / purpose_total))

defaulted, purpose_total = couples_default_status(bank_data, singles, 'wedding')
if purpose_total > 0:
    print("\nTotal wedding loans sought: {:.2%}".format(purpose_total / single_total))
    print("Defaulted on wedding loan: {:.2%}".format(defaulted / purpose_total))
    print("Did not default on wedding loan: {:.2%}".format((purpose_total-defaulted) / purpose_total))
else:
    print("\nSurprisingly, the singles were not spending money on weddings;"
          + " married people would likely be single when getting a loan for a wedding.")

----------------------------------------
STATS FOR COUPLES
Total loans sought by couples: 16462
Ratio of loans sought by couples to total loans sought: 76.92%
Ratio of default by couples to loans sought by couples: 7.97%

Total real estate loans sought: 19.94%
Defaulted on real estate loan: 7.56%
Did not default on real estate loan: 92.44%

Total education loans sought: 46.70%
Defaulted on education loan: 7.86%
Did not default on education loan: 92.14%

Total car loans sought: 19.17%
Defaulted on car loan: 8.78%
Did not default on car loan: 91.22%

Total wedding loans sought: 14.20%
Defaulted on wedding loan: 7.83%
Did not default on wedding loan: 92.17%

----------------------------------------
STATS FOR SINGLES
Total loans sought by singles: 4940
Ratio of loans sought by singles to total loans sought: 23.08%
Ratio of default by singles to loans sought by singles: 8.50%

Total real estate loans sought: 23.64%
Defaulted on real estate loan: 7.45%
Did not default on real estate loan: 92

- After processing the data and a cursory analysis, we can surmise several interesting facts:
    - If a low income, widow/widower wants a loan for one of her/his 5 children, then they are very likely to not default.
    - If a medium income, married person with no children wants a loan for education, then lender beware.
    
- Possible caveats for observed phenomena:
    - When looking at the raw totals, it appeas that couples tend to default more on their loans. However, putting these numbers in context reveals that the opposite is true.
    - Couples account for nearly 77% of borrowers and thus we find that their total defaulters are greater in number than the singles group. However, the couples default on nearly 8% of their loans; whereas, the singles default on 8.5% of theirs.
    - When comparing the different purposes for loans, we find that couples default approx. 1% more than singles on real estate loans, 1% less on education loans, and approx. 2% less on car loans. Somewhat surprising was that no singles took out loans for weddings, despite presumably being unmarried, divorced, or widower/widowered when paying for a wedding. 